# **Clasificador de Imágenes de Flores**

## Librerías.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import skimage as ski
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, make_scorer

## Importación del Dataset

In [4]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        directory = '/content/drive/My Drive/ProyectoIA/flowers',
        target_size=(100, 100),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        directory = '/content/drive/My Drive/ProyectoIA/flowers',
        target_size=(100, 100),
        batch_size=32,
        class_mode='binary')

Found 11792 images belonging to 16 classes.
Found 11792 images belonging to 16 classes.


**Verificación del archivo.**

In [5]:
base_dir =  r'/content/drive/My Drive/ProyectoIA/flowers'
folders = os.listdir(base_dir)
print(folders)

['california_poppy', 'dandelion', 'coreopsis', 'daffodil', 'common_daisy', 'carnation', 'astilbe', 'black_eyed_susan', 'calendula', 'bellflower', 'tulip', 'magnolia', 'water_lily', 'sunflower', 'rose', 'iris']


## Cargar imágenes a cada categoría.

In [6]:
# Listas de nombres de archivo para cada categoría
water_lily = sorted(os.listdir(os.path.join(base_dir, 'water_lily')))
tulip = sorted(os.listdir(os.path.join(base_dir, 'tulip')))
sunflower = sorted(os.listdir(os.path.join(base_dir, 'sunflower')))
rose = sorted(os.listdir(os.path.join(base_dir, 'rose')))
magnolia = sorted(os.listdir(os.path.join(base_dir, 'magnolia')))
iris = sorted(os.listdir(os.path.join(base_dir, 'iris')))
dandelion = sorted(os.listdir(os.path.join(base_dir, 'dandelion')))
daffodil = sorted(os.listdir(os.path.join(base_dir, 'daffodil')))
coreopsis = sorted(os.listdir(os.path.join(base_dir, 'coreopsis')))
common_daisy = sorted(os.listdir(os.path.join(base_dir, 'common_daisy')))
carnation = sorted(os.listdir(os.path.join(base_dir, 'carnation')))
california_poppy = sorted(os.listdir(os.path.join(base_dir, 'california_poppy')))
calendula = sorted(os.listdir(os.path.join(base_dir, 'calendula')))
black_eyed_susan = sorted(os.listdir(os.path.join(base_dir, 'black_eyed_susan')))
bellflower = sorted(os.listdir(os.path.join(base_dir, 'bellflower')))
astilbe = sorted(os.listdir(os.path.join(base_dir, 'astilbe')))

## Redes Neuronales



**Tratamiento de imágenes**

In [36]:
data = []  # Características
labels = []  # Etiquetas

# Define las categorías y etiquetas
categorias = {
    'california_poppy': 0,
    'dandelion': 1,
    'coreopsis': 2,
    'daffodil': 3,
    'common_daisy': 4,
    'carnation': 5,
    'astilbe': 6,
    'black_eyed_susan': 7,
    'calendula': 8,
    'bellflower': 9,
    'tulip': 10,
    'magnolia': 11,
    'water_lily': 12,
    'sunflower': 13,
    'rose': 14,
    'iris': 15
}

# Itera sobre las categorías y carga las imágenes
for categoria, label in categorias.items():
    try:
        carpeta = base_dir + '/' + categoria + '/'
        for imagen in os.listdir(carpeta):
            img = cv2.imread(carpeta + imagen)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(img, (64, 64))
            data.append(np.array(resized))
            labels.append(label)
    except AttributeError:
        print('')

Permutación de las imágenes.

In [8]:
imagenes = np.array(data)
labels = np.array(labels)

#Permutación
p = np.random.permutation(range(len(imagenes)))
imagenes = imagenes[p]
labels = labels[p]

División de los datos 80/20

In [10]:
X_train, X_test, y_train, y_test = train_test_split(imagenes, labels, test_size = 0.2, random_state = 31)

X_train = X_train.reshape([-1, 64, 64, 3])
X_test = X_test.reshape([-1, 64, 64, 3])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_test = tf.keras.utils.to_categorical(y_test)
y_train = tf.keras.utils.to_categorical(y_train)

X_train /= 255
X_test /= 255
print('x_train shape:', X_train.shape)
print('Numero de imagenes en x_train', X_train.shape[0], "y_train:", y_train.shape)
print('Numero de imagenes in x_test', X_test.shape[0], "y_test: ", y_test.shape)

x_train shape: (9433, 64, 64, 3)
Numero de imagenes en x_train 9433 y_train: (9433, 16)
Numero de imagenes in x_test 2359 y_test:  (2359, 16)


**capa entrada, capa oculta, capa salida**

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),  # Changed input shape to match X_train
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="softmax")
])

# Use 'categorical_crossentropy' for one-hot encoded labels
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
model.fit(X_train, y_train, epochs=10, verbose=0)  # 10 epochs
y_pred_probs = model.predict(X_test)  # Get predicted probabilities
y_pred = y_pred_probs.argmax(axis=1)  # Convert probabilities to class labels

# Now both y_test and y_pred are in the same format (class labels)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 2ms/step
Accuracy: 0.0564


In [15]:
model.fit(X_train, y_train, epochs=20, verbose=0)  # 20 epochs
y_pred_probs = model.predict(X_test)  # Get predicted probabilities
y_pred = y_pred_probs.argmax(axis=1)  # Convert probabilities to class labels

# Now both y_test and y_pred are in the same format (class labels)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 5ms/step
Accuracy: 0.0564


In [16]:
model.fit(X_train, y_train, epochs=30, verbose=0)  # 30 epochs
y_pred_probs = model.predict(X_test)  # Get predicted probabilities
y_pred = y_pred_probs.argmax(axis=1)  # Convert probabilities to class labels

# Now both y_test and y_pred are in the same format (class labels)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 3ms/step
Accuracy: 0.0564


**capa entrada, (3)capa oculta, capa salida**

In [60]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="softmax")  # Output layer for 16 classes
])

# If y_train is one-hot encoded, use 'categorical_crossentropy'
model2.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [61]:
model2.fit(X_train, y_train, epochs=10, verbose=0)
y_pred_probs = model2.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# No need to convert y_test if it's already one-hot encoded
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 3ms/step
Accuracy: 0.2921


In [62]:
model2.fit(X_train, y_train, epochs=20, verbose=0)
y_pred_probs = model2.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# No need to convert y_test if it's already one-hot encoded
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 5ms/step
Accuracy: 0.3306


In [63]:
model2.fit(X_train, y_train, epochs=30, verbose=0)
y_pred_probs = model2.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# No need to convert y_test if it's already one-hot encoded
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 5ms/step
Accuracy: 0.2942


**capa entrada, (6) capas ocultas, capa salida**

In [64]:
model3 = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(64, 64, 3)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),  # Primera capa oculta con 64 neuronas
    tf.keras.layers.Dense(32, activation="relu"),  # Segunda capa oculta con 32 neuronas
    tf.keras.layers.Dense(32, activation="relu"),  # Tercera capa oculta con 32 neuronas
    tf.keras.layers.Dense(16, activation="relu"),  # Cuarta capa oculta con 16 neuronas
    tf.keras.layers.Dense(16, activation="relu"),  # Quinta capa oculta con 16 neuronas
    tf.keras.layers.Dense(16, activation="relu"),  # Sexta capa oculta con 16 neuronas
    tf.keras.layers.Dense(16, activation="softmax")  # Capa de salida con 16 clases
])

model3.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [65]:
model3.fit(X_train, y_train, epochs=10, verbose=0)
y_pred_probs = model2.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# No need to convert y_test if it's already one-hot encoded
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 5ms/step
Accuracy: 0.2942


In [66]:
model3.fit(X_train, y_train, epochs=20, verbose=0)
y_pred_probs = model2.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# No need to convert y_test if it's already one-hot encoded
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 3ms/step
Accuracy: 0.2942


In [67]:
model3.fit(X_train, y_train, epochs=30, verbose=0)
y_pred_probs = model2.predict(X_test)
y_pred = y_pred_probs.argmax(axis=1)

# No need to convert y_test if it's already one-hot encoded
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f"Accuracy: {accuracy:.4f}")

74/74 [==============================] - 0s 3ms/step
Accuracy: 0.2942
